pip installs

In [ ]:
#pip install requests

imports

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from sklearn.preprocessing import StandardScaler, LabelEncoder
# from sklearn.model_selection import train_test_split 
# from sklearn import datasets, tree
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score, recall_score, precision_score
%matplotlib inline
import plotly.graph_objects as go
import plotly.io as pio  # me permite dibujar en un notebook
pio.renderers.default = "notebook"  # me permite dibujar en un notebook
import plotly.express as px
import plotly.figure_factory as ff
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.model_selection import GridSearchCV
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import itertools
import requests
import os

Load csv

In [122]:
rawdf = pd.read_csv("C:/Users/ggrinberg/Downloads/QR6129X1.csv")  # carga csv
#agregar busqueda de "Parameter" y generacion de exception en caso de no hallarlo así no se pierde en caso de no haber
Parameter_col_value=(rawdf.iloc[:,0]=='Parameter').idxmax()+1  # busca la primera fila cn el nombre "Parametro", si o encuentra nada devuelve 1 
df = pd.read_csv("C:/Users/ggrinberg/Downloads/QR6129X1.csv", header=Parameter_col_value)


In [164]:
general_data = rawdf.iloc[10:30,[0,1]]  # global data extracted by position
general_data = general_data.set_index(general_data.iloc[:, 0]).iloc[:,1]
general_data = general_data.to_dict()
HardBinDict = (rawdf[rawdf.iloc[:,0]=='HardBinName'].iloc[:,[1,2]])
HardBinDict = HardBinDict.set_index(HardBinDict.iloc[:, 0]).iloc[:,1]
HardBinDict = HardBinDict.to_dict()
SoftBinDict = (rawdf[rawdf.iloc[:,0]=='SoftBinName'].iloc[:,[1,2]])
SoftBinDict = SoftBinDict.set_index(SoftBinDict.iloc[:, 0]).iloc[:,1]
SoftBinDict = SoftBinDict.to_dict()
dftestdata = df[[' SBIN','HBIN','DIE_X','DIE_Y','SITE','TIME','TOTAL_TESTS','LOT_ID','WAFER_ID']]
df.drop(columns=[' SBIN','HBIN','DIE_X','DIE_Y','SITE','TIME','TOTAL_TESTS','LOT_ID','WAFER_ID'])

,Parameter,VCC1 OS Pre_D1,VCC1 OS Pre_D2,PWM OS Pre_D1,PWM OS Pre_D2,CSN OS Pre_D1,CSN OS Pre_D2,SCLK OS Pre_D1,SCLK OS Pre_D2,MOSI OS Pre_D1,...,MOSI OS Pre_D2.1,MISO OS Pre_D1.1,MISO OS Pre_D2.1,AU OS Pre_D1.1,AU OS Pre_D2.1,BV OS Pre_D1.1,BV OS Pre_D2.1,IW OS Pre_D1.1,IW OS Pre_D2.1,Unnamed: 822
0,Tests#,5000001,5000002,5000005,5000006,5000007,5000008,5000009,5000010,5000011,...,310000012,310000013,310000014,310000015,310000016,310000017,310000018,310000019,310000020,NaN
1,Patterns,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Unit,VOLTS,VOLTS,VOLTS,VOLTS,VOLTS,VOLTS,VOLTS,VOLTS,VOLTS,...,VOLTS,VOLTS,VOLTS,VOLTS,VOLTS,VOLTS,VOLTS,VOLTS,VOLTS,NaN
3,HighL,-18,-18,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,...,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,NaN
4,LowL,NaN,NaN,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,...,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,PID-141,-18.62479,-19.77288,-0.5596977,-0.5604649,-0.5338187,-0.5350795,-0.5341371,-0.535323,-0.534418,...,-0.5196635,-0.5192277,-0.5197197,-0.515313,-0.5161232,-0.5178229,-0.5184833,-0.5188718,-0.5200194,
137,PID-142,-18.62823,-19.79903,-0.5594055,-0.5597637,-0.5330882,-0.5352105,-0.5331256,-0.5351544,-0.5333504,...,-0.5200006,-0.5194525,-0.519832,-0.5152007,-0.5172096,-0.5174296,-0.518652,-0.5182725,-0.5194387,
138,PID-143,-18.62625,-19.80957,-0.5595029,-0.5603675,-0.5328072,-0.5345924,-0.5341371,-0.5352293,-0.5337999,...,-0.5203378,-0.519209,-0.5206187,-0.5154442,-0.5165728,-0.5177106,-0.5191015,-0.5185347,-0.5202067,
139,PID-144,-18.63741,-19.78137,-0.5594639,-0.5601922,-0.5339311,-0.5356039,-0.534493,-0.5352855,-0.533725,...,-0.5202816,-0.5199769,-0.5205626,-0.5160061,-0.5167039,-0.5176918,-0.5197384,-0.5181226,-0.5198508,


pendiente:
